## Import thư viện và khởi tạo OCR Engine

In [8]:
import paddle
print(paddle.__version__)
paddle.device.is_compiled_with_cuda()

3.2.0


True

In [9]:
import sys
sys.path.insert(0, '..')
import pathlib
PROJECT_ROOT = pathlib.Path().resolve().parents[1]
sys.path.append(str(PROJECT_ROOT))
import os
import json
import glob
import matplotlib.pyplot as plt
from PIL import Image

from src.ocr.ocr_processor import PaddleOCRProcessor

# Khởi tạo OCR processor
ocr_processor = PaddleOCRProcessor(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False
)

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/phn/.paddlex/official_models/PP-OCRv5_server_det`.


Đang khởi tạo PaddleOCR engine...


Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/phn/.paddlex/official_models/PP-OCRv5_server_rec`.


-> PaddleOCR đã sẵn sàng!


## 1.5. Test Token Classification

Token classifier phân loại OCR tokens TRƯỚC khi grouping:
- **Form fields**: "Name:", "Date:", "Address:"
- **Dates**: "12/01/2024", "Jan 12, 2024"
- **Numbers**: "$1,234.56", "1000"
- **Codes**: "ABC12345", "INV-001"

In [10]:
from src.ocr.token_classifier import TokenClassifier

# Initialize classifier
classifier = TokenClassifier()

# Test với sample tokens
test_tokens = [
    {'text': 'Invoice Number:', 'bbox': [10, 10, 120, 30]},
    {'text': 'INV-2024-001', 'bbox': [130, 10, 220, 30]},
    {'text': 'Date:', 'bbox': [10, 40, 60, 60]},
    {'text': '12/01/2024', 'bbox': [70, 40, 150, 60]},
    {'text': 'Total Amount:', 'bbox': [10, 70, 110, 90]},
    {'text': '$1,234.56', 'bbox': [120, 70, 200, 90]},
]

# Classify
classified = classifier.classify_tokens(test_tokens)

print("="*70)
print("TOKEN CLASSIFICATION DEMO")
print("="*70)

for token in classified:
    print(f"{token['text']:20} → {token['token_type']}")

# Statistics
stats = classifier.get_statistics(classified)
print(f"\n{'Statistics':20} → {stats}")
print("="*70)

TOKEN CLASSIFICATION DEMO
Invoice Number:      → form_key
INV-2024-001         → text
Date:                → form_key
12/01/2024           → form_value
Total Amount:        → form_key
$1,234.56            → number

Statistics           → {'form_key': 3, 'text': 1, 'form_value': 1, 'number': 1}


## Cấu hình đường dẫn

In [11]:
from src.utils.constant import *
from src.ocr.layout import *

## Chọn 1 ảnh mẫu

In [12]:
# Tìm 1 ảnh mẫu
import random

sample_image = '../dataset/DocVQA_Images\\train\\10668.png'
idx = random.randint(0, 1000)
if sample_image is None:
    for subset in SUBSETS:
        subset_folder = os.path.join(IMAGES_FOLDER, subset)
        if os.path.exists(subset_folder):
            images = glob.glob(os.path.join(subset_folder, "*.png"))[idx:idx+1]
            print(images)
            if images:
                sample_image = images[0]
                break
else:
    images = os.path.join(IMAGES_FOLDER, 'train', f'{sample_image}.png')
    
group(sample_image, ocr_processor)

Chạy OCR với Layout Analysis...


---
## Document Layout Analysis

Phân tích cấu trúc document sau khi OCR:
- **Stage I**: Group token → line → block
- **Stage II**: Detect regions (table, form, figure, text, layout)

## Visualize Stage I Groups

Hiển thị các bước grouping ở Stage I:
- **Token**: Các bounding box từ OCR
- **Line**: Gộp tokens thành lines
- **Block**: Gộp lines thành blocks

In [13]:
visualize_stage1_groups(sample_image, ocr_processor)

Chạy OCR với Layout Analysis...
Không có kết quả layout
